In [105]:
import math
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

# Лабораторная работа №2. Методы поиска

Вариант 12

## Задание 1 (15. Алгоритм имитации отжига)

### Классификация алгоритма

* По типу алгоритма: гибридный
* По устойчивости: неустойчивый
* По месту хранения данных: может хранить данные на месте или выделять доп. пространство
* По выделению дополнительного места: использует
* По адаптивности: может быть адаптивным, если использует изменяемые параметры. в остальных случаях - неадаптивный
* По сложности: 
    * Худшее значение: O(2^n)
    * Среднее значение: O(m*n)
    * Лучшее значение: O(log n)

### Описание алгоритма

Алгоритм имитации отжига (simulated annealing) — это стохастический метод оптимизации, используемый для нахождения глобального минимума или максимума некоторой функции. Определяется начальное решение и температура, выбирается случайное решение в окрестности текущего решения и рассчитывается изменение функции цели между текущим и новым решением. Если изменение положительное (новое решение хуже), то оно принимается с определенной вероятностью, зависящей от температуры. Если изменение отрицательное (новое решение лучше), то принимают его. Температура уменьшается по заранее заданной функции. Действия повторяются до тех пор, пока температура не станет равной нулю.

### Псевдокод

In [ ]:
current_state := initial_state()
current_energy := energy(current_state)
T := initial_temperature()
T_min := final_temperature()
L := iterations_per_temperature()
alpha := cooling_factor()
iteration_counter := 0
best_state := current_state
best_energy := current_energy

while T > T_min do
    for i in range(L) do
        new_state := random_move(current_state)
        new_energy := energy(new_state)
        if new_energy < current_energy do
            current_state := new_state
            current_energy := new_energy
        end if
        else do
            probability := exp((current_energy - new_energy) / T)
            if random() < probability do
                current_state := new_state
                current_energy := new_energy
            end if
    end for
    if current_energy < best_energy do
        best_state := current_state
        best_energy := current_energy
    end if
    T := T * alpha
    iteration_counter += L
end while

return best_state, best_energy

### Достоинства и недостатки

* Достоинства:
    * Может работать на любых типах задач и не требует знания градиентов
    * Является глобальной оптимизационной стратегией, что позволяет находить глобальные экстремумы в сложных функциях
    * Может работать в реальном времени и адаптироваться к изменениям в функции стоимости
* Недостатки:
    * Требуется настройка большого количества параметров, таких как начальная температура, скорость охлаждения и длина каждой итерации
    * Не гарантирует нахождение оптимального решения, только приближение к нему
    * Время работы может быть довольно длительным для сложных задач, особенно при больших размерностях

### Реализация сортировки

In [65]:
import math
import random

def simulated_annealing(initial_state, cost_func, neighbors_func, temperature, cooling_rate):
    current_state = initial_state
    current_cost = cost_func(current_state)
    best_state = current_state
    best_cost = current_cost
    while temperature > 0.1:
        neighbor_state = random.choice(neighbors_func(current_state))
        neighbor_cost = cost_func(neighbor_state)
        delta_cost = neighbor_cost - current_cost
        if delta_cost < 0 or math.exp(-delta_cost / temperature) > random.random():
            current_state = neighbor_state
            current_cost = neighbor_cost
        if current_cost < best_cost:
            best_state = current_state
            best_cost = current_cost
        temperature *= 1 - cooling_rate
    return best_state, best_cost

initial_state = [0, 1, 2, 3, 4]

# определение функции стоимости
def cost_func(state):
    return sum(state)

# определение функции соседних состояний
def neighbors_func(state):
    neighbors = []
    for i in range(len(state)):
        for j in range(i+1, len(state)):
            neighbor = state.copy()
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighbors.append(neighbor)
    return neighbors

# запуск алгоритма имитации отжига
best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 1000, 0.03)

print("Лучшее состояние:", best_state)
print("Лучшая стоимость:", best_cost)

Лучшее состояние: [0, 1, 2, 3, 4]
Лучшая стоимость: 10


### Тест

In [11]:
def test_simulated_annealing():
    def cost_func(x): # Тест на квадратичной функции
        return x**2
    
    def neighbors_func(x): # Тест на ф-ции соседних состояний
        return [x-1, x+1]

    initial_state = 5

    best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 100, 0.03)

    assert abs(best_state) < 0.1 #Ошибка при тестировании на квадратичной функции: best_state слишком далеко от минимума

    def cost_func(x): # Тест на ф-ции с множеством локальных минимумов
        return math.sin(5 * x) + math.sin(x)

    def neighbors_func(x): # Ф-ция соседних состояний
        return [x + random.uniform(-0.1, 0.1)]

    initial_state = random.uniform(-10, 10)

    best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 1000, 0.03)

    assert abs(best_state - (-1.39)) < 0.1 #Ошибка при тестировании на функции с множеством локальных минимумов: best_state слишком далеко от глобального минимума
    
    # Тест на задаче коммивояжера
    distances = [
        [0, 2, 5, 7],
        [2, 0, 6, 3],
        [5, 6, 0, 1],
        [7, 3, 1, 0]
    ]

    def cost_func(route): # Ф-ция стоимости
        cost = 0
        for i in range(len(route)):
            cost += distances[route[i-1]][route[i]]
        return cost

    def neighbors_func(route): # Ф-ция соседних состояний
        neighbors = []
        for i in range(len(route)):
            for j in range(i+1, len(route)):
                neighbor = route.copy()
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        return neighbors

    initial_state

### Поиск экстремума функции

In [61]:
import math
import random

def simulated_annealing(initial_state, cost_func, neighbors_func, temperature, cooling_rate):
    current_state = initial_state
    current_cost = cost_func(current_state)
    best_state = current_state
    best_cost = current_cost
    while temperature > 0.1:
        neighbor_state = random.choice(neighbors_func(current_state))
        neighbor_cost = cost_func(neighbor_state)
        delta_cost = neighbor_cost - current_cost
        if delta_cost < 0 or math.exp(-delta_cost / temperature) > random.random():
            current_state = neighbor_state
            current_cost = neighbor_cost
        if current_cost < best_cost:
            best_state = current_state
            best_cost = current_cost
        temperature *= 1 - cooling_rate
    return best_state, best_cost

# Определение ф-ции стоимости
def cost_func(x):
    return 12*x**3+12*x**2

# Определение ф-ции соседних состояний
def neighbors_func(x):
    delta = 0.1
    return [x + delta, x - delta]

# Запуск алгоритма имитации отжига
initial_state = 0
best_state, best_cost = simulated_annealing(initial_state, cost_func, neighbors_func, 1000, 0.03)

print("Лучшее состояние: ", best_state)
print("Лучшая стоимость: ", best_cost)

Лучшее состояние:  -9.699999999999982
Лучшая стоимость:  -9822.995999999941


## Задание 2 (4. Фибоначчиев поиск)

### Классификация алгоритма

* По типу алгоритма: гибридный
* По устойчивости: неустойчивый
* По месту хранения данных: не на месте
* По выделению дополнительного места: без выделения
* По адаптивности: адаптивный
* По сложности: 
    * Худшее значение: O(log n)
    * Среднее значение: O(log n)
    * Лучшее значение: O(log n) 

### Описание алгоритма

Метод поиска Фибоначчи аналогичен основной идее бинарного поиска, который заключается в уменьшении длины последовательности поиска и поиске ключевых слов (последующее число равно сумме двух предыдущих чисел). Его процесс поиска заключается в следующем: сначала определить область записи для поиска, а затем постепенно сужать область поиска, пока не найден интервал между двумя ключами, где может располагаться отыскиваемый ключ (поиск также может завершиться неудачно).

### Псевдокод

In [ ]:
function fibonacci_search(arr, x)
    fib_k := 0
    fib_k1 := 1
    fib_k2 := 1
    while fib_k2 < length(arr) do
        fib_k := fib_k1
        fib_k1 := fib_k2
        fib_k2 := fib_k + fib_k1

    offset := 0
    while fib_k > 0 do
        i := min(offset + fib_k1, length(arr)) - 1
        if arr[i] < x then
            fib_k := fib_k2 - fib_k1
            fib_k2 := fib_k1
            fib_k1 := fib_k - fib_k1
            offset := i
        else if arr[i] > x then
            fib_k := fib_k1 - fib_k
            fib_k1 := fib_k2 - fib_k1
            fib_k2 := fib_k - fib_k1
        else
            return i
    end while
    return -1
end function

### Реализация сортировки

In [70]:
def fibonacci_search(arr, x):
    fib_k = 0
    fib_k1 = 1
    fib_k2 = 1
    while fib_k2 < len(arr):
        fib_k = fib_k1
        fib_k1 = fib_k2
        fib_k2 = fib_k + fib_k1

    offset = 0
    while fib_k > 0:
        i = min(offset + fib_k1, len(arr)) - 1
        if arr[i] < x:
            fib_k = fib_k2 - fib_k1
            fib_k2 = fib_k1
            fib_k1 = fib_k - fib_k1
            offset = i
        elif arr[i] > x:
            fib_k = fib_k1 - fib_k
            fib_k1 = fib_k2 - fib_k1
            fib_k2 = fib_k - fib_k1
        else:
            return i
    return -1

### Тест

In [67]:
def test_fibonacci_search():
    arr = [1, 3, 4, 6, 7, 9, 11]
    
    # Проверяем, что элемент, присутствующий в массиве, находится правильно
    assert fibonacci_search(arr, 7) == 4
    
    # Проверяем, что элемент, отсутствующий в массиве, не находится
    assert fibonacci_search(arr, 5) == -1
    
    # Проверяем, что элемент в начале массива находится правильно
    assert fibonacci_search(arr, 1) == 0
    
    # Проверяем, что элемент в конце массива находится правильно
    assert fibonacci_search(arr, 11) == 6
    
    # Проверяем, что функция возвращает -1 при пустом массиве
    assert fibonacci_search([], 1) == -1

### Сравнение указанных алгоритмов поиска для массивов, содержащих n1, n2, n3, n4 элементов

In [104]:
import time
import random

arr_sizes = [1000, 5000, 10000, 100000]
for n in arr_sizes:
    arr = list(range(n))
    x = n - 1
    start_time = time.time()
    index = fibonacci_search(arr, x)
    end_time = time.time()
    elapsed_time = "{:.4f}".format(end_time - start_time)
    if index != -1:
        print("Значение", x, "найдено в индексе", index, "для массива размером", n, "за", elapsed_time, "секунд")
    else:
        print("Значение", x, "не найдено в массиве размером", n, "за", elapsed_time, "секунд")

Значение 999 найдено в индексе 999 для массива размером 1000 за 0.0000 секунд
Значение 4999 найдено в индексе 4999 для массива размером 5000 за 0.0000 секунд
Значение 9999 найдено в индексе 9999 для массива размером 10000 за 0.0000 секунд
Значение 99999 найдено в индексе 99999 для массива размером 100000 за 0.0000 секунд
